<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/05-LangChain_Redis/05.02_Dolly_v2_LangChain_Redis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Question Answering with LangChain, Dolly LLM and Redis

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

This notebook would use `databricks/dolly-v2-3b` LLM from HuggingFace, Redis with Vector Similarity Search and LangChain to answer questions about the information contained in a document.

### Install Dependencies

WARNING!!! We have to downgrade the version of one of the libraries here and that requires restart of the notebook runtime. At the end of the next cell you'll be prompted to restart the runtime. Restart and continue with the execution.

In [1]:
!pip install -q redis "langchain==0.0.227" tiktoken sentence_transformers transformers accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.5 MB/s eta 0:00:00


In [2]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import torch

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)



A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


### Install Redis Stack

Redis will be used as Vector Similarity Search engine for LangChain. Instead of using in-notebook Redis Stack you can provision your own free instance of Redis in the cloud. Get your own Free Redis Cloud instance at https://redis.io/try-free/ 

In [3]:
%%sh
# curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg   # COMMENTED OUT - Using Redis Cloud by default
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list 
sudo apt-get update  > /dev/null 2>&1
# # sudo apt-get install redis-stack-server  > /dev/null 2>&1  # COMMENTED OUT - Using Redis Cloud by default  # COMMENTED OUT - Using Redis Cloud by default
# redis-stack-server --daemonize yes   # COMMENTED OUT - Using Redis Cloud by default
print("✅ Using Redis Cloud - no local installation needed!")


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb focal main
Starting redis-stack-server, database path /var/lib/redis-stack


### Connect to Redis

By default this notebook would connect to the local instance of Redis Stack. If you have your own Redis Cloud instance - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [4]:
import redis
import os

# Redis Cloud Configuration (Default)
# Using Redis Cloud for persistent data storage

REDIS_HOST = os.getenv("REDIS_HOST", "redis-15306.c329.us-east4-1.gce.redns.redis-cloud.com")
REDIS_PORT = os.getenv("REDIS_PORT", "15306")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "ftswOcgMB8dLCnjbKMMDBg3DNnpi1KO5")
REDIS_USERNAME = os.getenv("REDIS_USERNAME", "default")
#Replace values above with your own if using Redis Cloud instance
# Example: REDIS_HOST="redis-15306.c329.us-east4-1.gce.redns.redis-cloud.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

#shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test Redis connection
!redis-cli $REDIS_CONN PING
print(f"🔗 Connecting to Redis Cloud: {REDIS_HOST}:{REDIS_PORT}")


In [5]:
from langchain.vectorstores.redis import Redis
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [6]:
!wget https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt

--2023-06-12 18:23:04--  https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39027 (38K) [text/plain]
Saving to: ‘state_of_the_union.txt’

state_of_the_union. 100%[===================>]  38.11K  --.-KB/s    in 0.003s  

2023-06-12 18:23:04 (12.8 MB/s) - ‘state_of_the_union.txt’ saved [39027/39027]



### Load text and split it into manageable chunks

Without this step any large body of text would exceed the limit of tokens you can feed to the LLM

In [7]:
from langchain.document_loaders import TextLoader
loader = TextLoader("./state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=30)
texts = text_splitter.split_documents(documents)

### Initialize embeddings engine

Here we are using https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2 which is a compact embedding engine that can be run locally. For large number of embeddings/documents you might consider GPU - enabled runtime or calling the hosted Embeddings API.

In [8]:
def get_embeddings():
  from langchain.embeddings import HuggingFaceEmbeddings
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
  return embeddings

embeddings = get_embeddings()

### Initialize LLM

In this notebook we are using Databricks Dolly-v2-3b LLM that we loaded earlier

In [9]:
#llm = OpenAI()
llm = HuggingFacePipeline(pipeline=generate_text)

### Create vector store from the documents using Redis as Vector Database

In [10]:
def get_vectorstore() -> Redis:
    """Create the Redis vectorstore."""

    try:
        vectorstore = Redis.from_existing_index(
            embedding=embeddings,
            index_name=INDEX_NAME,
            redis_url=REDIS_URL
        )
        return vectorstore
    except:
        pass

    # Load Redis with documents
    vectorstore = Redis.from_documents(
        documents=texts,
        embedding=embeddings,
        index_name=INDEX_NAME,
        redis_url=REDIS_URL
    )
    return vectorstore


redis = get_vectorstore()

## Specify the prompt template

PromptTemplate defines the exact text of the response that would be fed to the LLM. This step is optional, but the defaults usually work well for OpenAI and might fall short for other models.

In [11]:
def get_prompt():
    """Create the QA chain."""
    from langchain.prompts import PromptTemplate
    from langchain.chains import RetrievalQA

    # Define our prompt
    prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, say that you don't know, don't try to make up an answer.

    This should be in the following format:

    Question: [question here]
    Answer: [answer here]

    Begin!

    Context:
    ---------
    {context}
    ---------
    Question: {question}
    Answer:"""

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    return prompt

### Putting it all together

This is where the LangChain brings all the components together in a form of a simple QnA chain

In [12]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=redis.as_retriever(),
    #return_source_documents=True,
    chain_type_kwargs={"prompt": get_prompt()}
    )

### Debugging Redis

The code block below is example of how you can interact with the Redis Database

In [13]:
#!redis-cli $REDIS_CONN keys "*"
#!redis-cli $REDIS_CONN HGETALL "doc:qna:idx:063955c855a7436fbf9829821332ed2a"

###-- FLUSHDB will wipe out the entire database!!! Use with caution --###
#!redis-cli $REDIS_CONN flushdb


## Finally - let's ask questions! 

Examples:
- What did the president say about Ketanji Brown Jackson
- Did he mention Stephen Breyer?
- What was his stance on Ukraine

In [14]:


query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

'\nPresident Obama: "One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence."'